In [1]:
import pandas as pd
from datetime import date

In [2]:
DATOS = pd.read_csv('./INPUT/DEFINITIVAS-C3-29-09.csv', skiprows=1, sep=';', encoding='latin-1')
CANCELADOS = pd.read_excel('./INPUT/ESTUDIANTES A MATRICULAR CICLO 4.xlsx', engine='openpyxl', sheet_name='DESERTORES REPORTADOS')['CODIGO_UIS'].unique()
#CANCELADOS = pd.read_excel('./INPUT/CANCELADOS.xlsx', engine='openpyxl')['CODIGO_UIS'].unique()
EXCLUIR_CANCELADOS_ANTERIORES = True
if EXCLUIR_CANCELADOS_ANTERIORES:
    CANCELADOS_ANTERIORES = pd.read_excel('./INPUT/CANCELADOS ANTERIORES.xlsx', engine='openpyxl')['CODIGO_UIS'].unique()

REINGRESADOS = [
    2222985,
    2220048,
    2220040
]

CODIGOS = DATOS['ESTUDIANTE'].unique()
CICLO = 3
# La llave debe ser el nombre de la asignatura y el contenido una tupla (porcentaje, haceParteParcial),
# donde haceParteParcial es Verdadero si y solo si se requiere en el calculo individual de notas parciales.
# por defecto debe ser Verdadero si la asignatura NO es la asignatura con retos/sprints.
ASIGNATURAS = {
    'DESARROLLO DE SOFTWARE': (0.8, False),
    'INGLES III': (0.1, True),
    'COACHING III': (0.1, True)
}
UMBRAL_REPROBADO_SA = 2.945
UMBRAL_REPROBADO_100K = 3
REDONDEO_DECIMALES_SA = 1
REDONDEO_DECIMALES_100K = 2
INCLUIR_RETOS = True
if INCLUIR_RETOS:
    ASIGNATURA_RETOS = 'DESARROLLO DE SOFTWARE'
    DATOS_PARCIALES = pd.read_csv('./INPUT/PARCIALES-C3-29-09.csv', skiprows=1, sep=';', encoding='latin-1')
# Los porcentajes de la siguiente lista NO deben ser los de Moodle sino los dados por el MinTIC/Operador para EL CICLO
RETOS = [
    ('SPRINT_1', 0.15),
    ('SPRINT_2', 0.15),
    ('SPRINT_3', 0.25),
    ('SPRINT_4', 0.25),
]
N_RETOS = len(RETOS)

In [3]:
rta = {
    'CÓDIGO': [],
    'NOMBRE': [],
    'APELLIDO': [],
    'DEFINITIVA_REAL_SA': [],
    'DEFINITIVA_CICLO_SA': [],
    'REPROBÓ_SA': [],
    'CANCELÓ_MATRICULA': []
}
for asignatura in ASIGNATURAS:    
    rta[asignatura] = []
if INCLUIR_RETOS:
    for reto, _ in RETOS:
        rta[reto] = []
    rta['DEFINITIVA_100K'] = []
    rta['REPROBÓ_100K'] = []
    rta['SA_IGUAL_A_100K'] = []
    rta['DIFERENCIA_DEFSA_DEF100K'] = []

for cod in CODIGOS:    
    reingreso = cod in REINGRESADOS
    if EXCLUIR_CANCELADOS_ANTERIORES and cod in CANCELADOS_ANTERIORES and not (reingreso):
        continue
    cancelado = cod in CANCELADOS and not(reingreso)
    dfDefinitiva = DATOS[DATOS['ESTUDIANTE'] == cod]
    acum = 0
    acumP = 0
    for index, row in dfDefinitiva.iterrows():
        asignaturaActual = row['ASIGNATURA']
        rta[asignaturaActual].append(row['NOTA'])
        nota = row['NOTA'] * ASIGNATURAS[asignaturaActual][0]
        acum += nota
        acumP += nota if ASIGNATURAS[asignaturaActual][1] else 0
    
    definitiva = round(acum, REDONDEO_DECIMALES_SA)
    repSA = True if definitiva < UMBRAL_REPROBADO_SA else False
    rta['CÓDIGO'].append(cod)
    rta['NOMBRE'].append(row['NOMBRES'])
    rta['APELLIDO'].append(row['APELLIDOS'])
    rta['DEFINITIVA_REAL_SA'].append(acum)
    rta['DEFINITIVA_CICLO_SA'].append(definitiva)
    rta['REPROBÓ_SA'].append(repSA)
    rta['CANCELÓ_MATRICULA'].append(cancelado)
    
    if INCLUIR_RETOS:
        dfParciales = DATOS_PARCIALES[(DATOS_PARCIALES['ESTUDIANTE'] == cod) & (DATOS_PARCIALES['ASIGNATURA'] == ASIGNATURA_RETOS)]
        if len(dfParciales) != N_RETOS:
            print ('Error en número de actividades parciales', cod)
        for i in range(N_RETOS):
            reto, porcentaje = RETOS[i]
            notaP = dfParciales[dfParciales['No'] == (i+1)]['NOTA'].iloc[0]
            rta[reto].append(notaP)
            acumP += notaP * porcentaje
        def100k = round(acumP, REDONDEO_DECIMALES_100K)
        rep100k = True if def100k < UMBRAL_REPROBADO_100K else False
        rta['DEFINITIVA_100K'].append(def100k)
        rta['REPROBÓ_100K'].append(rep100k)
        rta['SA_IGUAL_A_100K'].append(repSA == rep100k)
        rta['DIFERENCIA_DEFSA_DEF100K'].append(definitiva - def100k)

In [5]:
# Generación y guardado Dataset definitivas
rCompleto = pd.DataFrame(rta)
ordenRCompleto = [
    'CÓDIGO',
    'NOMBRE',
    'APELLIDO',
    'CANCELÓ_MATRICULA'
]
ordenRCompleto += list(ASIGNATURAS.keys())
if INCLUIR_RETOS:
    ordenRCompleto += [reto[0] for reto in RETOS]
ordenRCompleto += ['DEFINITIVA_REAL_SA', 'DEFINITIVA_CICLO_SA', 'REPROBÓ_SA']
if INCLUIR_RETOS:
    ordenRCompleto += ['DEFINITIVA_100K', 'REPROBÓ_100K', 'SA_IGUAL_A_100K', 'DIFERENCIA_DEFSA_DEF100K']
rCompleto = rCompleto[ordenRCompleto]
rCompleto.to_excel('./OUTPUT/DEFINITIVAS_CICLO_' + str(CICLO) + '_' + str(date.today()) + '.xlsx', index = False)

# Generación y guardado Dataset nivelatorios
rNivelar = rCompleto[(rCompleto['REPROBÓ_SA'] == True) & (rCompleto['CANCELÓ_MATRICULA'] == False)]
rNivelar.to_excel('./OUTPUT/ESTUDIANTES_NIVELAR_CICLO_' + str(CICLO) + '_' + str(date.today()) + '.xlsx', index = False)